In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

In [4]:
class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()

In [40]:
C_IN = 5
D_MODEL = 5

# tsor = torch.tensor(list(range(C_IN)))
# emb_obj = FixedEmbedding(c_in=C_IN,d_model=D_MODEL)

In [70]:
w = torch.zeros(C_IN, D_MODEL).float()
w.require_grad = False

position = torch.arange(0, C_IN).float().unsqueeze(1)
div_term = (torch.arange(0, D_MODEL, 2).float() * -(math.log(10000.0) / D_MODEL)).exp()

w[:, 0::2] = torch.sin(position * div_term)[:w[:, 0::2].shape[0],:w[:, 0::2].shape[1]]
w[:, 1::2] = torch.cos(position * div_term)[:w[:, 1::2].shape[0],:w[:, 1::2].shape[1]]

# emb = nn.Embedding(C_IN, D_MODEL)
# emb.weight = nn.Parameter(w, requires_grad=False)

In [73]:
conv_layer = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=1)

In [78]:
tst_input = torch.tensor([1,2,3,4]).float().reshape((1,-1))

conv_layer(tst_input)

tensor([[0.3960, 0.3152, 0.2343, 0.1534]], grad_fn=<SqueezeBackward1>)

In [79]:
conv_layer.kernel_size

(1,)

In [22]:

m = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0)
input = torch.tensor([[1, 2, 3, 4, 5]], dtype=torch.float32)
output = m(input)
output

tensor([[ 0.4709,  0.6300,  0.7892],
        [-2.1107, -3.2507, -4.3907]], grad_fn=<SqueezeBackward1>)

In [19]:
input = np.array([[1, 2, 3, 4, 5],[1, 2, 3, 4, 5]])#.reshape(1,-1)
kernel_filt = m.weight.detach().numpy()[0][0]
kernel_bias =m.bias.detach().numpy()
k_size=3
out_lst = []
for n_i,arry in enumerate(input):
    temp_out = []
    for i in range(len(arry)):
        slc_lst = arry[i:i+k_size]
        if len(slc_lst)==k_size:
            temp_out.append(np.sum(slc_lst*kernel_filt)+kernel_bias[n_i])

    out_lst.append(temp_out)
    # break
out_lst


[[1.6338245868682861, 2.2841799557209015, 2.934535324573517],
 [1.7435266077518463, 2.3938819766044617, 3.044237345457077]]

In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_emb, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_emb).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_emb, 2).float() * -(math.log(10000.0) / d_emb)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)[:pe[:, 0::2].shape[0],:pe[:, 0::2].shape[1]]
        pe[:, 1::2] = torch.cos(position * div_term)[:pe[:, 1::2].shape[0],:pe[:, 1::2].shape[1]]

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(2)]

In [64]:
max_len = 2
d_emb = 4

pe = torch.zeros(max_len, d_emb).float()
pe.require_grad = False

position = torch.arange(0, max_len).float().unsqueeze(1)
div_term = (torch.arange(0, d_emb, 2).float() * -(math.log(10000.0) / d_emb)).exp()

pe[:, 0::2] = torch.sin(position * div_term)#[:pe[:, 0::2].shape[0],:pe[:, 0::2].shape[1]]
pe[:, 1::2] = torch.cos(position * div_term)#[:pe[:, 1::2].shape[0],:pe[:, 1::2].shape[1]]

# pe = pe.unsqueeze(0)

In [66]:
pe.T

tensor([[0.0000, 0.8415],
        [1.0000, 0.5403],
        [0.0000, 0.0100],
        [1.0000, 0.9999]])

In [68]:
print(torch.sin(torch.tensor(1)))
print(torch.cos(torch.tensor(1/(10**2))))
print(torch.sin(torch.tensor(1/10**4)))
print(torch.cos(torch.tensor(1/10**6)))

tensor(0.8415)
tensor(0.9999)
tensor(1.0000e-04)
tensor(1.)


In [2]:


seq_len = 2
d_emb = 4
POS_EMB = torch.zeros(d_emb, seq_len).float()
POS = torch.arange(0, seq_len).repeat(d_emb,1)
DIV = 10**(4*torch.arange(0, d_emb).reshape(-1,1).repeat(1,seq_len)*2/d_emb)

POS_EMB[0::2, :] = torch.sin(POS/DIV)[0::2, :]
POS_EMB[1::2, :] = torch.cos(POS/DIV)[1::2, :]

In [3]:
POS_EMB

tensor([[0.0000e+00, 8.4147e-01],
        [1.0000e+00, 9.9995e-01],
        [0.0000e+00, 1.0000e-04],
        [1.0000e+00, 1.0000e+00]])